# 任务说明



In [2]:
import os

import numpy as np
import pandas as pd

import torch
from torch import nn

## 数据集介绍

### 车流量数据

数据由铺设在道路上的检测线圈采集。

原始数据使用numpy二进制文件存储，可以使用numpy.load函数读取。下面简单讲解数据的读取方式，以及数据的含义。

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.utils.data as data
import warnings
warnings.filterwarnings("ignore")

In [3]:
raw_data = np.load('../Datasets/traffic-flow/traffic.npz')['data']
print(raw_data.shape)
print(raw_data)

(17856, 170, 3)
[[[1.330e+02 6.030e-02 6.580e+01]
  [2.100e+02 5.890e-02 6.960e+01]
  [1.240e+02 3.580e-02 6.580e+01]
  ...
  [7.400e+01 2.131e-01 6.530e+01]
  [9.400e+01 2.260e-02 6.800e+01]
  [6.000e+00 3.100e-03 6.500e+01]]

 [[1.140e+02 5.320e-02 6.690e+01]
  [1.850e+02 5.500e-02 6.850e+01]
  [1.190e+02 3.390e-02 6.500e+01]
  ...
  [7.300e+01 1.469e-01 3.720e+01]
  [8.400e+01 1.890e-02 6.870e+01]
  [4.000e+00 1.800e-03 6.500e+01]]

 [[1.400e+02 6.220e-02 6.680e+01]
  [1.710e+02 4.660e-02 6.990e+01]
  [1.070e+02 3.360e-02 6.380e+01]
  ...
  [7.000e+01 5.860e-02 3.400e+01]
  [8.200e+01 2.200e-02 6.700e+01]
  [4.000e+00 2.100e-03 6.490e+01]]

 ...

 [[1.200e+02 5.810e-02 6.330e+01]
  [1.760e+02 5.290e-02 6.680e+01]
  [1.190e+02 5.180e-02 5.610e+01]
  ...
  [4.700e+01 1.551e-01 3.220e+01]
  [9.100e+01 2.290e-02 6.640e+01]
  [3.000e+00 1.400e-03 6.530e+01]]

 [[1.020e+02 5.790e-02 6.140e+01]
  [1.650e+02 4.920e-02 6.720e+01]
  [1.330e+02 5.070e-02 5.890e+01]
  ...
  [9.700e+01 1.265e-01

In [4]:
print(raw_data.shape)

(17856, 170, 3)


可以看到数据包含3个维度。

- 第一个维度代表数据的总条数。也就是说，原始长序列总长为16992.
- 第二个维度代表传感器的数量。
- 第三个维度代表每个传感器收集到的不同种类数据，分别为车流量、拥挤程度和车速。在实际实验中，可以任意选取一个特征进行预测。

综上所述，本数据集包含170个传感器，每个传感器都包含长度为17856的连续采样值序列。在构造训练集时，可首先将每个传感器的长序列划分为训练/验证/测试序列，再使用滑动窗口。本数据的时间轴**已经被规整化且无任何缺失值，因此仅需使用固定长度的滑动窗口**。

    **在这个实验中，可以只选一个传感器，通过window_size步数据预测下一时间段的值，下面是构造训练集的示例代码：**

In [1]:
target = 0       # 选择第一维数据进行预测
window_size = 12
sensor_num = 5      # 选择5号感器

train_x = []
train_y = []
len_train = int(raw_data.shape[0] * 0.6)
train_seqs = raw_data[:len_train]
for i in range(train_seqs.shape[0] - window_size):
    train_x.append(train_seqs[i:i+window_size, sensor_num, :].squeeze())
    train_y.append(train_seqs[i+window_size, sensor_num, target].squeeze())

train_x = torch.Tensor(train_x)
train_y = torch.Tensor(train_y)
train_x.shape

NameError: name 'raw_data' is not defined

### FourSquare checkin数据集

FourSquare是一个地点推荐网站，类似于国内的大众点评。当用户到达某个地点时，可以通过手机App进行“签到”(check-in)，如此一来，将一个用户所有的签到记录按照时间顺序排序，就能得到此用户的行动轨迹。本实验中使用的数据包含纽约的用户签到数据，存储在FS_NYC.csv文件中。原始数据直接通过逗号分隔值(csv)格式存储，可以通过pandas进行读取和简单的处理。

In [6]:
nyc_data = pd.read_csv(os.path.join('../dataset', 'foursquare-checkin', 'FS_NYC.csv'), parse_dates=[-1])
nyc_data.head()

,userId,venueId,venueCategoryId,venueCategory,latitude,longitude,timezoneOffset,utcTimestamp
0,470,49bbd6c0f964a520f4531fe3,4bf58dd8d48988d127951735,Arts & Crafts Store,40.719810,-74.002581,-240,2012-04-03 18:00:09+00:00
1,979,4a43c0aef964a520c6a61fe3,4bf58dd8d48988d1df941735,Bridge,40.606800,-74.044170,-240,2012-04-03 18:00:25+00:00
2,69,4c5cc7b485a1e21e00d35711,4bf58dd8d48988d103941735,Home (private),40.716162,-73.883070,-240,2012-04-03 18:02:24+00:00
3,395,4bc7086715a7ef3bef9878da,4bf58dd8d48988d104941735,Medical Center,40.745164,-73.982519,-240,2012-04-03 18:02:41+00:00
4,87,4cf2c5321d18a143951b5cec,4bf58dd8d48988d1cb941735,Food Truck,40.740104,-73.989658,-240,2012-04-03 18:03:00+00:00


数据包含的列，以及它们的含义：

| 列名             | 含义 |
|-----------------|------|
| userId          |用户ID，不同用户的唯一标识符|
| venueId         |地点ID，不同地点的唯一标识符|
| venueCategoryId |地点类别ID|
| venueCategory   |地点类别的名称|
|latitude|地点的纬度|
|longitude|地点的经度|
|timezoneOffset|所在时区相对于格林威治时间的时差(分钟)|
|utcTimestamp|格林威治标准时间|

在本次实验中，我们暂时只需要使用userId, venueId和utcTimestamp这三列。每位用户的所有check-in记录可以看作是一条序列。借助pandas中的groupby函数，我们可以遍历每位用户的所有记录。

| 列名             | 含义 |
|-----------------|------|
| userId          |用户ID，不同用户的唯一标识符|
| venueId         |地点ID，不同地点的唯一标识符|
| venueCategoryId |地点类别ID|
| venueCategory   |地点类别的名称|
|latitude|地点的纬度|
|longitude|地点的经度|
|timezoneOffset|所在时区相对于格林威治时间的时差(分钟)|
|utcTimestamp|格林威治标准时间|

注意到原始的地点标识符venueId是hash码，这种格式是难以输入到神经网络模型中的。为了后续输入模型的方便，我们最好在预处理时将venueId映射到为从0开始的类标签。

> 通过pandas.DataFrame.drop_duplicates()函数，我们可以得到包含数据集中所有venueId且无重复的集合。构造一个`{venue_id: venue_index}`的映射(dict)，然后调用pandas.Series.map()即可快速将原始的hash码变为类别标签。

In [8]:
venue_id2index = {id:index for index, id in enumerate(nyc_data['venueId'].drop_duplicates())}
nyc_data['venueIndex'] = nyc_data['venueId'].map(venue_id2index)
nyc_data.head()

,userId,venueId,venueCategoryId,venueCategory,latitude,longitude,timezoneOffset,utcTimestamp,venueIndex
0,470,49bbd6c0f964a520f4531fe3,4bf58dd8d48988d127951735,Arts & Crafts Store,40.719810,-74.002581,-240,2012-04-03 18:00:09+00:00,0
1,979,4a43c0aef964a520c6a61fe3,4bf58dd8d48988d1df941735,Bridge,40.606800,-74.044170,-240,2012-04-03 18:00:25+00:00,1
2,69,4c5cc7b485a1e21e00d35711,4bf58dd8d48988d103941735,Home (private),40.716162,-73.883070,-240,2012-04-03 18:02:24+00:00,2
3,395,4bc7086715a7ef3bef9878da,4bf58dd8d48988d104941735,Medical Center,40.745164,-73.982519,-240,2012-04-03 18:02:41+00:00,3
4,87,4cf2c5321d18a143951b5cec,4bf58dd8d48988d1cb941735,Food Truck,40.740104,-73.989658,-240,2012-04-03 18:03:00+00:00,4


可以看到新增的列`venueIndex`即为我们需要的类别标签。

下面是构造训练集的示例代码。需要注意的是，由于check-in数据集时间轴的不规整性，实际上使用**固定时间跨度**的滑动窗口更加合理。本任务的目的是通过前window_size个轨迹点来预测下一个轨迹点。

In [9]:
window_size = 12

train_x = []
train_y = []
for user_id, group in nyc_data.groupby('userId'):
    # pandas会对userId进行遍历。
    # 每次遍历中，group包含了对应userId所有的check-in记录。
    user_trajectory = group.sort_values(['utcTimestamp'])['venueIndex'].tolist()
    train_trajectory = user_trajectory[:int(len(user_trajectory) * 0.6)]
    for i in range(len(train_trajectory) - window_size):
        train_x.append(train_trajectory[i:i+window_size])
        train_y.append(train_trajectory[i+window_size])

train_x = np.array(train_x)
print(train_x.shape)

(123027, 12)


#### 额外说明：使用RNN处理离散数据

另外需要说明的是，由于类标签是离散数据，直接输入RNN模型是不合理的。一般的做法是使用一组可学习参数，将每个地点映射到一个一定长度的**嵌入向量**（Embedding vector）中。PyTorch中可以简单地实现：

In [10]:
# 取一个batch的原始轨迹序列
batch = torch.from_numpy(train_x[:64]).long()
print('原始序列：', batch.shape)

# 初始化嵌入层。嵌入向量的长度需要指定，且input_size与其相等。
embed_size = 128
embed_layer = nn.Embedding(len(venue_id2index), embed_size)
embedded_batch = embed_layer(batch)
print('嵌入后序列：', embedded_batch.shape)

原始序列： torch.Size([64, 12])
嵌入后序列： torch.Size([64, 12, 128])


可以看到嵌入后的序列符合RNN的输入格式，只不过每条数据的特征维度不再是1，而是嵌入向量的长度。需要注意的是嵌入层的参数是随机初始化的，应当与RNN共同训练。

轨迹预测任务实际上就是**分类任务**，预测模型的原始输出应当为(batch_size, number_of_locations)，即每条输出均包含对数据集中可能出现的地点的概率的预测，可以用一个线性模型作为“预测头”。Loss函数一般使用CrossEntropyLoss，评估函数可选择Accuracy，Recall或F1_score。

In [11]:
# 训练过程示例代码
rnn_model = nn.RNN(input_size=embed_size, hidden_size=128, num_layers=2, batch_first=True)
output_model = nn.Linear(128, len(venue_id2index))

loss_func = nn.CrossEntropyLoss()

rnn_out, _ = rnn_model(embedded_batch[:, :-1, :])
prediction = output_model(rnn_out[:, -1, :])
label = batch[:, -1]
print(loss_func(prediction, label))

tensor(10.5921, grad_fn=<NllLossBackward0>)


In [12]:
# 评估过程示例代码
from sklearn.metrics import accuracy_score, recall_score, f1_score

pre_d = prediction.argmax(-1).detach().cpu().numpy()
label_d = label.detach().cpu().numpy()
acc, rec, f1_micro, f1_macro = accuracy_score(label_d, pre_d), \
                               recall_score(label_d, pre_d, average='macro'), \
                               f1_score(label_d, pre_d, average='micro'), \
                               f1_score(label_d, pre_d, average='macro')

D:\applications\miniconda\lib\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
